# Introduction to JAX

This script will present the main functions from JAX that will be used during the research project, in order to optimize the code.

## Packages

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random

import utils as ut

## Trying JIT and Grad function

In [2]:
# Creating an array to test jit.
seed = 0
key = random.PRNGKey(seed)
data = random.normal(key, (1000000,))

# Defining a random function.
def f(x):
    return jnp.exp(x)**2 / jnp.exp(3) * jnp. exp(x**2)

# Using jit function.
f_jit = jit(f)

#ploting the elapsed time.
print('non-jit version:')
%timeit f(data)
print('jit version:')
%timeit f_jit(data)

# the more complicated the function, the higher it is the difference between f and f_jit.

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


non-jit version:
4.52 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
jit version:
530 µs ± 61.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


grad of function f(x):

In [3]:
# The use of lambda functions makes it easier to use grad().
g = lambda x: jnp.exp(x)**2 + x**4
x = jnp.arange(0.,5.,0.5)

# the grad functions from jax only works for scalar functions.
g_grad = grad(g)

def grad_jax(x):   
    h = jnp.zeros(len(x))
    for i in range(0, len(x)):
        h = h.at[i].set(g_grad(x[i]))
    return h

grad_jax_jit = jit(grad_jax)

%timeit grad_jax(x)
%timeit grad_jax_jit(x)

51.8 ms ± 5.92 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
2.07 µs ± 68.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


grad of function f(x,y):

In [4]:
g = lambda x, y: 5*x*y + 7*y
x = jnp.arange(0.,5.,1.)
y = jnp.arange(0.,10.,2.)

print(x)
print(y)

g_grad_x = grad(g)
g_grad_y = grad(g, argnums=(1))

# Function that computes the grad of a function in regards of x.
def grad_x(x):   
    h = jnp.zeros((np.size(x), np.size(y)))
    for i in range(0, np.size(x)):
        for j in range(0, np.size(y)):
            h = h.at[i,j].set(g_grad_x(x[i],y[j]))
    return h

# Function that computes the grad of a function in regards of y.
def grad_y(x):   
    h = jnp.zeros((np.size(x), np.size(y)))
    for i in range(0, np.size(x)):
        for j in range(0, np.size(y)):
            h = h.at[i,j].set(g_grad_y(x[i],y[j]))
    return h


print(grad_x(x))
print(grad_y(y))

# The function grad computes the derivative of a scalar function at one point, if we have multiple points, another
# function that will compute the grad for each point of the array had to be implemented. That is not optimum since
# it is good to avoid loops. VMAP function will be disussed.


[0. 1. 2. 3. 4.]
[0. 2. 4. 6. 8.]
[[ 0. 10. 20. 30. 40.]
 [ 0. 10. 20. 30. 40.]
 [ 0. 10. 20. 30. 40.]
 [ 0. 10. 20. 30. 40.]
 [ 0. 10. 20. 30. 40.]]
[[ 7.  7.  7.  7.  7.]
 [17. 17. 17. 17. 17.]
 [27. 27. 27. 27. 27.]
 [37. 37. 37. 37. 37.]
 [47. 47. 47. 47. 47.]]


## Using VMAP function

In [6]:
# When using vmap, it is necessary to implement a function to treat a single data. Instead of creating another function
# with a loop, vmap allow the treatment of batches of data from a single data function.

f = lambda x: 5*x**2
x = jnp.arange(0.,5.,1.)

# grad function is a single data function.
f_grad = grad(f)
print(f_grad(x[1]))

# Using the functions above
print(derivative_jax(f,x[1],0))

# Function without using vmap.
def grad_loop(x):   
    h = jnp.zeros(len(x))
    for i in range(0, len(x)):
        h = h.at[i].set(f_grad(x[i]))
    return h      

# Function using vmap.
def vmap_grad(x):
    return vmap(f_grad)(x)

print('time elapsed using a loop:')
%timeit grad_loop(x)

print('time elapsed using vmap:')
%timeit vmap_grad(x)

jit_vmap_grad = jit(vmap_grad)

print('time elapsed using jit and vmap:')
%timeit jit_vmap_grad(x)

10.0


NameError: name 'derivative_jax' is not defined

In [7]:
z = np.random.rand(5,5)
z

array([[0.78477019, 0.67353179, 0.76777419, 0.80046085, 0.57403526],
       [0.38272031, 0.70147264, 0.28258596, 0.14359417, 0.80835885],
       [0.65309193, 0.16236008, 0.20778387, 0.78611871, 0.56364002],
       [0.0521202 , 0.24408649, 0.25093291, 0.75566982, 0.01124928],
       [0.47312528, 0.53067697, 0.66654804, 0.08060171, 0.33573778]])

In [8]:
z = grad(z)

TypeError: Expected a callable value, got [[0.78477019 0.67353179 0.76777419 0.80046085 0.57403526]
 [0.38272031 0.70147264 0.28258596 0.14359417 0.80835885]
 [0.65309193 0.16236008 0.20778387 0.78611871 0.56364002]
 [0.0521202  0.24408649 0.25093291 0.75566982 0.01124928]
 [0.47312528 0.53067697 0.66654804 0.08060171 0.33573778]]